이 colab 노트는 AI End to End 과정을 실습하기 위해 만들어짐 (동대문 Sesac)

# 사전 준비

## 필요한 package 들을 설치

In [ ]:
!apt install tree -y -qq

## 테스트에 사용할 Dataset 을 다운로드 후 train-data 폴더에 압축을 푼다

In [ ]:
# dataset download
!wget -q https://github.com/max5982/ES/releases/download/simson/gray-400-50-simpson.zip
!wget -q https://github.com/max5982/ES/releases/download/simson/gray-400-400-simpson.zip
!wget -q https://github.com/max5982/ES/releases/download/simson/gray-test-simpson.zip
!wget -q https://github.com/max5982/ES/releases/download/simson/gray-800-simpson.zip

# 압축 풀기
!unzip -q gray-400-50-simpson.zip -d train-data
!unzip -q gray-400-400-simpson.zip -d train-400-data
!unzip -q gray-800-simpson.zip -d train-800-data
!unzip -q gray-test-simpson.zip -d test-data

# 다운로드 받은 zip 파일 삭제
!rm gray-400-50-simpson.zip
!rm gray-400-400-simpson.zip
!rm gray-test-simpson.zip
!rm gray-800-simpson.zip

# 잘 압축이 풀렸는지 tree 명령어로 Level 2 단계까지 확인
!tree -L 2

In [ ]:
# 각 dataset 의 갯수를 확인
!find ./train-data/bart_simpson/ -type f | wc -l
!find ./train-data/homer_simpson/ -type f | wc -l
!find ./train-data/lisa_simpson/ -type f | wc -l
!find ./train-data/marge_simpson/ -type f | wc -l

# 첫번째 Practice - 2개의 hidden layer 를 가진 ANN model

## 모델 정의

In [ ]:
import numpy as np

# ReLU 활성화 함수
def relu(x):
    return np.maximum(0, x)

# ReLU의 미분 함수 (역전파용)
def relu_deriv(x):
    return (x > 0).astype(float)

# Softmax 활성화 함수 (출력층에 사용)
def softmax(x):
    e = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e / np.sum(e, axis=1, keepdims=True)

# 가중치 및 편향 초기화 함수
def initialize_weights(input_size, hidden1, hidden2, output_size):
    params = {
        # 입력층 → 첫 번째 은닉층 가중치/편향
        "W1": np.random.randn(input_size, hidden1) * 0.1,
        "b1": np.zeros((1, hidden1)),
        # 첫 번째 은닉층 → 두 번째 은닉층 가중치/편향
        "W2": np.random.randn(hidden1, hidden2) * 0.1,
        "b2": np.zeros((1, hidden2)),
        # 두 번째 은닉층 → 출력층 가중치/편향
        "W3": np.random.randn(hidden2, output_size) * 0.1,
        "b3": np.zeros((1, output_size)),
    }
    return params

# 순전파(Forward Pass) 함수
def forward_pass(x, params):
    # 입력층 → 첫 번째 은닉층
    z1 = x @ params["W1"] + params["b1"]
    a1 = relu(z1)
    # 첫 번째 은닉층 → 두 번째 은닉층
    z2 = a1 @ params["W2"] + params["b2"]
    a2 = relu(z2)
    # 두 번째 은닉층 → 출력층
    z3 = a2 @ params["W3"] + params["b3"]
    out = softmax(z3)
    # 순전파 중간 결과 저장 (역전파용)
    cache = {"x": x, "z1": z1, "a1": a1, "z2": z2, "a2": a2, "z3": z3, "out": out}
    return out, cache

# 역전파(Backpropagation) 및 파라미터 업데이트 함수
def backward_pass(params, cache, y_true, lr, sample_weights=None):
    m = y_true.shape[0] # 데이터 샘플 개수
    dz3 = cache["out"] - y_true # 출력층 오차(softmax + cross-entropy)

    # 샘플별 가중치 적용 (옵션)
    if sample_weights is not None:
        sample_weights = sample_weights.reshape(-1, 1)
        dz3 *= sample_weights

    # 두 번째 은닉층 → 출력층 가중치/편향의 기울기
    dW3 = cache["a2"].T @ dz3 / m
    db3 = np.sum(dz3, axis=0, keepdims=True) / m

    # 출력층 → 두 번째 은닉층 오차 전파
    da2 = dz3 @ params["W3"].T
    dz2 = da2 * relu_deriv(cache["z2"])
    dW2 = cache["a1"].T @ dz2 / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    # 두 번째 은닉층 → 첫 번째 은닉층 오차 전파
    da1 = dz2 @ params["W2"].T
    dz1 = da1 * relu_deriv(cache["z1"])
    dW1 = cache["x"].T @ dz1 / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    # 가중치 및 편향 업데이트 (경사하강법)
    params["W3"] -= lr * dW3
    params["b3"] -= lr * db3
    params["W2"] -= lr * dW2
    params["b2"] -= lr * db2
    params["W1"] -= lr * dW1
    params["b1"] -= lr * db1

## 모델 Training

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
from PIL import Image

def preprocess_image(img_path):
    img = Image.open(img_path).convert("L").resize((28, 28))
    return np.array(img).flatten().astype(np.float32) / 255.0

def one_hot_encode(label_index, num_classes):
    one_hot = np.zeros(num_classes)
    one_hot[label_index] = 1.0
    return one_hot

def load_dataset(base_path):
    X, y = [], []
    label_map = {}
    for idx, label_name in enumerate(sorted(os.listdir(base_path))):
        label_map[label_name] = idx
        folder = os.path.join(base_path, label_name)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            x = preprocess_image(img_path)
            X.append(x)
            y.append(one_hot_encode(idx, len(os.listdir(base_path))))
    return np.array(X), np.array(y), label_map

def train(X_train, y_train, params, lr, epochs):
    losses, accuracies = [], []
    for epoch in range(epochs):
        out, cache = forward_pass(X_train, params)
        loss = -np.sum(y_train * np.log(out + 1e-9)) / y_train.shape[0]
        preds = np.argmax(out, axis=1)
        labels = np.argmax(y_train, axis=1)
        acc = np.mean(preds == labels)

        losses.append(loss)
        accuracies.append(acc)

        backward_pass(params, cache, y_train, lr)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f} - Accuracy: {acc:.4f}")

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), losses, label='Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.title('Training Loss')

    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), accuracies, label='Accuracy', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.title('Training Accuracy')

    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.show()
    plt.close()

    return params

In [ ]:
np.random.seed(42)

X_train, y_train, label_map = load_dataset("train-data")
input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 128, 64, output_size)

# lr - learning rate
# epochs - 몇번 훈련할지 설정
params = train(X_train, y_train, params, lr=0.01, epochs=50)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

# 두번째 Practice - Training 데이터셋의 분할과 Early Stop

## Training 하는 부분 재정의
 - patience: 몇번이나 성능개선 없는지 count MAX
 - delta: 유의미한 성능 개선의 범위
 - test_size: Training : Validation ratio

In [ ]:
from sklearn.model_selection import train_test_split

def train(X_train, y_train, X_val, y_val, params, lr, epochs, patience=5, delta=0.001):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_loss = np.inf
    best_params = {k: v.copy() for k, v in params.items()}
    no_improve = 0

    for epoch in range(epochs):
        out_train, cache_train = forward_pass(X_train, params)
        train_loss = -np.sum(y_train * np.log(out_train + 1e-9)) / y_train.shape[0]
        train_preds = np.argmax(out_train, axis=1)
        train_acc = np.mean(train_preds == np.argmax(y_train, axis=1))

        backward_pass(params, cache_train, y_train, lr)

        out_val, _ = forward_pass(X_val, params)
        val_loss = -np.sum(y_val * np.log(out_val + 1e-9)) / y_val.shape[0]
        val_preds = np.argmax(out_val, axis=1)
        val_acc = np.mean(val_preds == np.argmax(y_val, axis=1))

        if val_loss < best_loss - delta:
            best_loss = val_loss
            best_params = {k: v.copy() for k, v in params.items()}
            no_improve = 0
            print(f"★ Validation improved to {val_loss:.4f}")
        else:
            no_improve += 1

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
              f"No improve: {no_improve}/{patience}")

        # Early Stop
        if no_improve >= patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1}!")
            params = {k: v.copy() for k, v in best_params.items()}
            break

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss', marker='o', color='orange')
    plt.plot(val_losses, label='Val Loss', marker='x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.title('Training vs Validation Loss')

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Acc', marker='o', color='orange')
    plt.plot(val_accuracies, label='Val Acc', marker='x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.title('Training vs Validation Accuracy')

    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.show()
    plt.close()

    return params

## 모델 Training

In [ ]:
np.random.seed(42)

X_full, y_full, label_map = load_dataset("train-data")
y_labels = np.argmax(y_full, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    # Training dataset split
    test_size=0.2, # 8:2 -> train:val ratio
    stratify=y_labels,
    random_state=42
)

input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 128, 64, output_size)

params = train(
    X_train, y_train, X_val, y_val,
    params, lr=0.01, epochs=100,
    patience=5, delta=0.0015
)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

# 세번째 Practice - Training 데이터셋의 불균형을 해결하기 위한 가중치 조절

## Training 하는 부분 재정의


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

def train(X_train, y_train, X_val, y_val, params, lr, epochs, patience=5, delta=0.001, class_weights=None):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_loss = np.inf
    best_params = {k: v.copy() for k, v in params.items()}
    no_improve = 0

    if class_weights is not None:
        class_weights = np.array(class_weights)
        sample_weights = class_weights[np.argmax(y_train, axis=1)]

    for epoch in range(epochs):
        out_train, cache_train = forward_pass(X_train, params)

        if class_weights is not None:
            loss_per_sample = -np.sum(y_train * np.log(out_train + 1e-9), axis=1)
            train_loss = np.sum(loss_per_sample * sample_weights) / y_train.shape[0]
        else:
            train_loss = -np.sum(y_train * np.log(out_train + 1e-9)) / y_train.shape[0]

        train_preds = np.argmax(out_train, axis=1)
        train_acc = np.mean(train_preds == np.argmax(y_train, axis=1))

        # dataset 에 가중치를 부여하는 부분
        if class_weights is not None:
            backward_pass(params, cache_train, y_train, lr, sample_weights)
        else:
            backward_pass(params, cache_train, y_train, lr)

        out_val, _ = forward_pass(X_val, params)
        val_loss = -np.sum(y_val * np.log(out_val + 1e-9)) / y_val.shape[0]
        val_preds = np.argmax(out_val, axis=1)
        val_acc = np.mean(val_preds == np.argmax(y_val, axis=1))

        if val_loss < best_loss - delta:
            best_loss = val_loss
            best_params = {k: v.copy() for k, v in params.items()}
            no_improve = 0
            print(f"★ Validation improved to {val_loss:.4f}")
        else:
            no_improve += 1

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
              f"No improve: {no_improve}/{patience}")

        if no_improve >= patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1}!")
            params = {k: v.copy() for k, v in best_params.items()}
            break

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss', marker='o', color='orange')
    plt.plot(val_losses, label='Val Loss', marker='x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.title('Training vs Validation Loss')

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Acc', marker='o', color='orange')
    plt.plot(val_accuracies, label='Val Acc', marker='x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.title('Training vs Validation Accuracy')

    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.show()
    plt.close()

    return params

## 모델 Training

In [ ]:
np.random.seed(42)

X_full, y_full, label_map = load_dataset("train-data")
y_labels = np.argmax(y_full, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    # Training dataset split
    test_size=0.2, # 8:2 -> train:val ratio
    stratify=y_labels,
    random_state=42
)

# dataset 의 불균형을 해소하기 위한 weight 값을 통한 가중치 부여
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = weights.astype(np.float32)
print(f"\nClass Weights: {class_weights}\n")

input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 128, 64, output_size)

params = train(
    X_train, y_train, X_val, y_val,
    params, lr=0.01, epochs=100,
    patience=5, delta=0.0015,
    class_weights=class_weights
)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

# 네번째 Practice - balanced dataset 을 충분한 양으로 늘려줬을 경우

## 400 개의 balanced dataset


In [ ]:
# 각 dataset 의 갯수를 확인
!find ./train-400-data/bart_simpson/ -type f | wc -l
!find ./train-400-data/homer_simpson/ -type f | wc -l
!find ./train-400-data/lisa_simpson/ -type f | wc -l
!find ./train-400-data/marge_simpson/ -type f | wc -l

## 모델 Training - 400/400/400/400

In [ ]:
np.random.seed(42)

X_full, y_full, label_map = load_dataset("train-400-data")
y_labels = np.argmax(y_full, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    # Training dataset split
    test_size=0.2, # 8:2 -> train:val ratio
    stratify=y_labels,
    random_state=42
)

# dataset 의 불균형을 해소하기 위한 weight 값을 통한 가중치 부여
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = weights.astype(np.float32)
print(f"\nClass Weights: {class_weights}\n")

input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 128, 64, output_size)

params = train(
    X_train, y_train, X_val, y_val,
    params, lr=0.01, epochs=100,
    patience=5, delta=0.0015,
    class_weights=class_weights
)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인 - 400/400/400/400

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

In [ ]:
from google.colab import files

files.download('model.pkl')

## 800 개의 balanced dataset


In [ ]:
# 각 dataset 의 갯수를 확인
!find ./train-800-data/bart_simpson/ -type f | wc -l
!find ./train-800-data/homer_simpson/ -type f | wc -l
!find ./train-800-data/lisa_simpson/ -type f | wc -l
!find ./train-800-data/marge_simpson/ -type f | wc -l

## 모델 Training - 800/800/800/800

In [ ]:
np.random.seed(42)

X_full, y_full, label_map = load_dataset("train-800-data")
y_labels = np.argmax(y_full, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    # Training dataset split
    test_size=0.2, # 8:2 -> train:val ratio
    stratify=y_labels,
    random_state=42
)

# dataset 의 불균형을 해소하기 위한 weight 값을 통한 가중치 부여
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = weights.astype(np.float32)
print(f"\nClass Weights: {class_weights}\n")

input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 128, 64, output_size)

params = train(
    X_train, y_train, X_val, y_val,
    params, lr=0.01, epochs=100,
    patience=5, delta=0.0015,
    class_weights=class_weights
)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인 - 800/800/800/800

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

# 다섯번째 Practice - Batch size 적용

## Training 하는 부분 재정의


In [ ]:
def train(X_train, y_train, X_val, y_val, params, lr, epochs, patience=5, delta=0.001, class_weights=None, batch_size=64):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_loss = np.inf
    best_params = {k: v.copy() for k, v in params.items()}
    no_improve = 0

    if class_weights is not None:
        sample_weights = np.array([class_weights[cls] for cls in np.argmax(y_train, axis=1)])

    for epoch in range(epochs):
        indices = np.random.permutation(X_train.shape[0])
        X_shuffled = X_train[indices]
        y_shuffled = y_train[indices]
        if class_weights is not None:
            sw_shuffled = sample_weights[indices]
        else:
            sw_shuffled = None

        # 배치로 나누기
        n_batches = X_train.shape[0] // batch_size
        if X_train.shape[0] % batch_size != 0:
            n_batches += 1

        epoch_train_loss = 0.0
        epoch_train_acc = 0.0

        for i in range(n_batches):
            start = i * batch_size
            end = min((i+1)*batch_size, X_train.shape[0])
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            if sw_shuffled is not None:
                sw_batch = sw_shuffled[start:end]
            else:
                sw_batch = None

            # Forward pass
            out_batch, cache_batch = forward_pass(X_batch, params)

            if class_weights is not None:
                loss_per_sample = -np.sum(y_batch * np.log(out_batch + 1e-9), axis=1)
                batch_loss = np.sum(loss_per_sample * sw_batch) / X_batch.shape[0]
            else:
                batch_loss = -np.sum(y_batch * np.log(out_batch + 1e-9)) / X_batch.shape[0]

            batch_preds = np.argmax(out_batch, axis=1)
            batch_acc = np.mean(batch_preds == np.argmax(y_batch, axis=1))

            if class_weights is not None:
                backward_pass(params, cache_batch, y_batch, lr, sw_batch)
            else:
                backward_pass(params, cache_batch, y_batch, lr)

            epoch_train_loss += batch_loss * X_batch.shape[0]
            epoch_train_acc += batch_acc * X_batch.shape[0]

        train_loss = epoch_train_loss / X_train.shape[0]
        train_acc = epoch_train_acc / X_train.shape[0]

        out_val, _ = forward_pass(X_val, params)
        val_loss = -np.sum(y_val * np.log(out_val + 1e-9)) / y_val.shape[0]
        val_preds = np.argmax(out_val, axis=1)
        val_acc = np.mean(val_preds == np.argmax(y_val, axis=1))

        if val_loss < best_loss - delta:
            best_loss = val_loss
            best_params = {k: v.copy() for k, v in params.items()}
            no_improve = 0
            print(f"★ Validation improved to {val_loss:.4f}")
        else:
            no_improve += 1

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
              f"No improve: {no_improve}/{patience}")

        if no_improve >= patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1}!")
            params = {k: v.copy() for k, v in best_params.items()}
            break

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss', marker='o', color='orange')
    plt.plot(val_losses, label='Val Loss', marker='x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.title('Training vs Validation Loss')

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Acc', marker='o', color='orange')
    plt.plot(val_accuracies, label='Val Acc', marker='x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.title('Training vs Validation Accuracy')

    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.show()
    plt.close()

    return params

## 모델 Training

In [ ]:
np.random.seed(42)

X_full, y_full, label_map = load_dataset("train-800-data")
y_labels = np.argmax(y_full, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    # Training dataset split
    test_size=0.2, # 8:2 -> train:val ratio
    stratify=y_labels,
    random_state=42
)

# dataset 의 불균형을 해소하기 위한 weight 값을 통한 가중치 부여
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = weights.astype(np.float32)
print(f"\nClass Weights: {class_weights}\n")

input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 128, 64, output_size)

params = train(
    X_train, y_train, X_val, y_val,
    params, lr=0.01, epochs=100,
    patience=5, delta=0.0015,
    class_weights=class_weights,
    # Batch Size 를 정의/적용
    batch_size=8
)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

# 여섯번째 Practice - hidden layer 추가

## Model 재정의

In [ ]:
def initialize_weights(input_size, hidden1, hidden2, hidden3, output_size):
    params = {
        "W1": np.random.randn(input_size, hidden1) * 0.1,
        "b1": np.zeros((1, hidden1)),
        "W2": np.random.randn(hidden1, hidden2) * 0.1,
        "b2": np.zeros((1, hidden2)),
        "W3": np.random.randn(hidden2, hidden3) * 0.1,
        "b3": np.zeros((1, hidden3)),
        "W4": np.random.randn(hidden3, output_size) * 0.1,
        "b4": np.zeros((1, output_size)),
    }
    return params

def forward_pass(x, params):
    z1 = x @ params["W1"] + params["b1"]
    a1 = relu(z1)
    z2 = a1 @ params["W2"] + params["b2"]
    a2 = relu(z2)
    z3 = a2 @ params["W3"] + params["b3"]
    a3 = relu(z3)
    z4 = a3 @ params["W4"] + params["b4"]
    out = softmax(z4)
    cache = {"x":x, "z1":z1, "a1":a1, "z2":z2, "a2":a2, "z3":z3, "a3":a3, "z4":z4, "out":out}
    return out, cache

def backward_pass(params, cache, y_true, lr, sample_weights=None):
    m = y_true.shape[0]
    dz4 = cache["out"] - y_true

    if sample_weights is not None:
       sample_weights = sample_weights.reshape(-1, 1)
       dz4 *= sample_weights

    dW4 = (cache["a3"].T @ dz4) / m
    db4 = np.sum(dz4, axis=0, keepdims=True) / m

    da3 = dz4 @ params["W4"].T
    dz3 = da3 * relu_deriv(cache["z3"])
    dW3 = (cache["a2"].T @ dz3) / m
    db3 = np.sum(dz3, axis=0, keepdims=True) / m

    da2 = dz3 @ params["W3"].T
    dz2 = da2 * relu_deriv(cache["z2"])
    dW2 = (cache["a1"].T @ dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    da1 = dz2 @ params["W2"].T
    dz1 = da1 * relu_deriv(cache["z1"])
    dW1 = (cache["x"].T @ dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    params["W4"] -= lr * dW4
    params["b4"] -= lr * db4
    params["W3"] -= lr * dW3
    params["b3"] -= lr * db3
    params["W2"] -= lr * dW2
    params["b2"] -= lr * db2
    params["W1"] -= lr * dW1
    params["b1"] -= lr * db1

## 모델 Training

In [ ]:
np.random.seed(42)

X_full, y_full, label_map = load_dataset("train-800-data")
y_labels = np.argmax(y_full, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full,
    # Training dataset split
    test_size=0.2, # 8:2 -> train:val ratio
    stratify=y_labels,
    random_state=42
)

# dataset 의 불균형을 해소하기 위한 weight 값을 통한 가중치 부여
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = weights.astype(np.float32)
print(f"\nClass Weights: {class_weights}\n")

input_size = X_train.shape[1]
output_size = y_train.shape[1]
params = initialize_weights(input_size, 512, 256, 128, output_size)

params = train(
    X_train, y_train, X_val, y_val,
    params, lr=0.01, epochs=100,
    patience=5, delta=0.0015,
    class_weights=class_weights,
    # Batch Size 를 정의/적용
    batch_size=8
)

with open("model.pkl", "wb") as f:
    pickle.dump((params, label_map), f)

## 훈련 결과 확인

In [ ]:
def load_images_in_folder(folder):
    data = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("L").resize((28, 28))
            arr = np.array(img).reshape(-1) / 255.0
            data.append(arr)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return np.array(data), filenames

def inference():
    with open("model.pkl", "rb") as f:
        params, label_map = pickle.load(f)

    id_to_label = {v: k for k, v in label_map.items()}

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        out, _ = forward_pass(X, params)
        preds = np.argmax(out, axis=1)
        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

In [ ]:
import pickle
from sklearn.metrics import classification_report

X_test, y_test, label_map = load_dataset("test-data")
with open("model.pkl", "rb") as f:
    params, label_map = pickle.load(f)

out, _ = forward_pass(X_test, params)
preds = np.argmax(out, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, preds, target_names=label_map.keys()))

# 일곱번째 Practice - Pytorch 사용하기

## 모델 정의

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.out = nn.Linear(hidden_sizes[2], output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)

def preprocess_image_tensor(img_path):
    img = Image.open(img_path).convert("L").resize((28, 28))
    img = torch.tensor(list(img.getdata()), dtype=torch.float32).view(1, 28, 28)
    return img.view(-1) / 255.0

def one_hot_encode_tensor(label_index, num_classes):
    return torch.nn.functional.one_hot(torch.tensor(label_index), num_classes).float()

def load_dataset_tensor(base_path):
    X, y = [], []
    label_map = {}
    class_names = sorted(os.listdir(base_path))
    for idx, label_name in enumerate(class_names):
        label_map[label_name] = idx
        folder = os.path.join(base_path, label_name)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            try:
                x = preprocess_image_tensor(img_path)
                X.append(x)
                y.append(one_hot_encode_tensor(idx, len(class_names)))
            except Exception as e:
                print(f"Failed to process {img_path}: {e}")
    return torch.stack(X), torch.stack(y), label_map

## 모델 Training

In [ ]:
%%time

import torch.optim as optim

def train_model(X_train, y_train, X_val, y_val, label_map, epochs=100, lr=0.001, patience=5, delta=0.0015, batch_size=8):
    print(f"** cuda? - {torch.cuda.is_available()} **")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_size = X_train.shape[1]
    output_size = y_train.shape[1]
    hidden_sizes = [512, 256, 128]

    model = DNN(input_size, hidden_sizes, output_size).to(device)

    labels = torch.argmax(y_train, dim=1)
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=lr)

    X_train = X_train.to(device)
    y_train = labels.to(device)
    X_val = X_val.to(device)
    y_val = torch.argmax(y_val, dim=1).to(device)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_model = None
    best_loss = float("inf")
    no_improve = 0

    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(X_train.size(0))
        total_loss, correct = 0.0, 0

        for i in range(0, X_train.size(0), batch_size):
            idx = permutation[i:i+batch_size]
            batch_x = X_train[idx]
            batch_y = y_train[idx]

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch_x.size(0)
            correct += (outputs.argmax(dim=1) == batch_y).sum().item()

        train_loss = total_loss / X_train.size(0)
        train_acc = correct / X_train.size(0)

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val).item()
            val_preds = val_outputs.argmax(dim=1)
            val_acc = (val_preds == y_val).float().mean().item()

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | "
              f"Val Loss={val_loss:.4f}, Acc={val_acc:.4f}")

        if val_loss < best_loss - delta:
            best_loss = val_loss
            best_model = model.state_dict()
            no_improve = 0
            print("★ Validation improved")
        else:
            no_improve += 1
            if no_improve >= patience:
                print("Early stopping!")
                break

    torch.save({'model_state_dict': best_model, 'label_map': label_map}, 'model_tensor.pth')

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label="Train Acc")
    plt.plot(val_accuracies, label="Val Acc")
    plt.legend()
    plt.grid()
    plt.savefig("training_metrics_tensor.png")
    plt.close()


X, y, label_map = load_dataset_tensor("train-800-data")
y_np = torch.argmax(y, dim=1).numpy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y_np)
train_model(X_train, y_train, X_val, y_val, label_map)

## 훈련 결과 확인

In [ ]:
def load_images_in_folder(folder):
    data, filenames = [], []
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            data.append(preprocess_image_tensor(path))
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")
    return torch.stack(data), filenames if data else (torch.empty(0), filenames)

def inference():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load("model_tensor.pth", map_location=device)
    label_map = checkpoint["label_map"]
    id_to_label = {v: k for k, v in label_map.items()}

    input_size = 28 * 28
    output_size = len(label_map)
    hidden_sizes = [512, 256, 128]

    model = DNN(input_size, hidden_sizes, output_size).to(device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()

    test_root = "test-data"
    total_correct, total_samples = 0, 0

    print("Detailed Inference Results\n==========================")
    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if X.numel() == 0:
            print(f"{class_name}: No images found.")
            continue

        X = X.to(device)
        outputs = model(X)
        preds = torch.argmax(outputs, dim=1).cpu()
        true_label = label_map[class_name]

        correct = 0
        print(f"\nClass: {class_name}\n" + "-"*40)
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred.item()]
            is_correct = pred.item() == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        acc = correct / len(preds) * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{len(preds)})")
        total_correct += correct
        total_samples += len(preds)

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

# 여덜번째 Practice - Transfer Learning

## Resnet50

In [ ]:
%%time

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from sklearn.metrics import accuracy_score
import pickle
import os

def get_dataloaders(data_dir, batch_size=32, val_ratio=0.2):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder(data_dir, transform=transform)
    class_names = dataset.classes

    val_size = int(len(dataset) * val_ratio)
    train_size = len(dataset) - val_size
    train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size)

    return train_loader, val_loader, class_names

def get_resnet50(num_classes):
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    #for param in model.parameters():
    #    param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    #for param in model.fc.parameters():
    #    param.requires_grad = True
    return model

def get_mobilenetv2(num_classes):
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

def train_model(data_dir="train-800-data", epochs=10, lr=1e-4, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("device {}".format(device))
    train_loader, val_loader, class_names = get_dataloaders(data_dir, batch_size)
    num_classes = len(class_names)

    model = get_resnet50(num_classes).to(device)
    #model = get_mobilenetv2(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    best_val_acc = 0.0
    for epoch in range(epochs):
        model.train()
        train_preds, train_labels = [], []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_acc = accuracy_score(train_labels, train_preds)

        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)
        print(f"Epoch {epoch+1}/{epochs} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_resnet50.pth")
            #torch.save(model.state_dict(), "best_mobilenetv2.pth")
            with open("class_names.pkl", "wb") as f:
                pickle.dump(class_names, f)

if __name__ == "__main__":
    train_model()

## 훈련결과 확인

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import pickle
from torchvision import models

def load_images_in_folder(folder):
    images = []
    filenames = []

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        try:
            img = Image.open(path).convert("RGB")
            img_tensor = transform(img)
            images.append(img_tensor)
            filenames.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    if images:
        images = torch.stack(images)  # Shape: (N, 3, 224, 224)
    else:
        images = torch.empty((0, 3, 224, 224))

    return images, filenames

def inference():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load class labels
    with open("class_names.pkl", "rb") as f:
        class_names = pickle.load(f)
    label_map = {name: idx for idx, name in enumerate(class_names)}
    id_to_label = {v: k for k, v in label_map.items()}
    num_classes = len(class_names)

    # Load model
    model = models.resnet50(weights=None)
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    model.load_state_dict(torch.load("best_resnet50.pth", map_location=device))
    #model.load_state_dict(torch.load("best_mobilenetv2.pth", map_location=device))
    model.to(device)
    model.eval()

    test_root = "test-data"
    total_correct = 0
    total_samples = 0

    print("Detailed Inference Results\n==========================")

    for class_name in sorted(os.listdir(test_root)):
        folder = os.path.join(test_root, class_name)
        if not os.path.isdir(folder): continue

        X, filenames = load_images_in_folder(folder)
        if len(X) == 0:
            print(f"\n{class_name}: No images found.\n")
            continue

        X = X.to(device)
        with torch.no_grad():
            outputs = model(X)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

        true_label = label_map[class_name]

        print(f"\nClass: {class_name}")
        print("-" * 40)
        correct = 0
        for i, pred in enumerate(preds):
            pred_label = id_to_label[pred]
            is_correct = pred == true_label
            mark = "✔️" if is_correct else "❌"
            print(f"{filenames[i]:25} → Predicted: {pred_label:15} {mark}")
            if is_correct:
                correct += 1

        total = len(preds)
        acc = correct / total * 100
        print(f"\n{class_name} Accuracy: {acc:.2f}% ({correct}/{total})")

        total_correct += correct
        total_samples += total

    print("\n==========================")
    print(f"Overall Accuracy : {total_correct / total_samples * 100:.2f}% ({total_correct}/{total_samples})")

if __name__ == "__main__":
    inference()

# 마무리

In [ ]:
!ls

In [ ]:
# 훈련해서 나온 pre-trained model 을 로컬로 다운로드 하기
from google.colab import files

files.download('best_resnet50.pth')
#files.download('best_mobilenetv2.pth')

* https://netron.app/ 에서 best_resnet50.pth 를 열어서 network 구조를 살펴보자